In [1]:
#import necessary libraries and frameworks
import gdspy
import pandas as pd
import numpy as np
import sys

import matplotlib.pylab as plt
import matplotlib.pyplot

from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import savetxt

In [2]:
#input parameters
baseUnit = 1 #layout scale(microns)
m=baseUnit*1e6 
nm = m*1E-9
mm = m*1e-3
scaling = 5; #scaling factor

periodX = 320*scaling*nm #period along x-axis
periodY = 320*scaling*nm #period along y-axis
Xmax = 0.25*mm #half of the metasurface length
Ymax = 0.25*mm #half of the metasurface width
Nx = 2*Xmax/periodX #number of meta-atoms along x-axis
Ny = 2*Ymax/periodY #number of meta-atoms along y-axis
phaseStep = 45 #phase step in degrees

In [3]:
#read phase map
# phaseMap = pd.read_csv("a-state_ideal.txt", header=None)
# phaseMap = phaseMap.values.tolist()

In [4]:
#generating analytical phase map
f = 2.5*mm #focal distance
lam = 680*nm #wavelength

In [5]:
#example of 1D ideal planar lens
x = np.linspace(-Xmax,Xmax,Nx)
phase_analyt0 = 2*np.pi/lam*(np.sqrt(x**2+f**2)-f)
phase_analyt = np.remainder(phase_analyt0,2*np.pi)

#plotting the phase dependence
plt.plot(x, phase_analyt)
plt.xlabel('x, um')
plt.ylabel('phase, rad')
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(20, 4)

TypeError: object of type <class 'float'> cannot be safely interpreted as an integer.

In [ ]:
#generating 2D phase map for an ideal planar lens
x = np.linspace(-Xmax,Xmax,Nx)
y = np.linspace(-Ymax,Ymax,Ny)
phase = []

X,Y = np.meshgrid(x,y)
phaseMap = np.rint(180/np.pi*np.remainder(2*np.pi/lam*(np.sqrt(X**2+Y**2+f**2)-f),2*np.pi)) #in degrees  

In [ ]:
savetxt('metasurface_phaseMap.csv', phaseMap, delimiter=',')


In [ ]:
#2D surface plot
fig = plt.figure()
plt.contourf(X,Y,phaseMap, cmap=cm.jet)
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(10, 10)

In [ ]:
#function to clean up cells
def clearCell(cell):
    cell.remove_polygons(lambda pts,layer,datatype:True)

In [ ]:
#layer and datatype tags
ld_Si = {"layer":1, "datatype":1}

In [ ]:
#creating a gdspy library
lib = gdspy.GdsLibrary()

In [ ]:
#MAPPING PHASE TO META-ATOM RADIUS
# phaseToRadiusMap = [[45+45*i,nm*(50+10*i)]for i in range(8)]
# phase to radius relations are taken from numerical simulations, vis WFOV a-Si metalens @ 680nm, height 800nm
phaseToRadius = [[0, 93*scaling*nm],[45, 87*scaling*nm], [90, 81.5*scaling*nm], [135,78*scaling*nm], [180,75*scaling*nm], [225, 59*scaling*nm], [270, 54.5*scaling*nm], [315, 46*scaling*nm]]
metaatomRadius = [93*scaling*nm, 87*scaling*nm, 81.5*scaling*nm, 78*scaling*nm, 75*scaling*nm, 59*scaling*nm, 54.5*scaling*nm, 46*scaling*nm]
phaseValue = [0, 45, 90, 135, 180, 225, 270, 315]

In [ ]:
#GENERATING META-ATOM lIBRARY
# disks = []
# for i in range(len(metaatomRadius)):
# disks.append(gdspy.Round((periodX*i, 0), metaatomRadius[i], number_of_points=12))  
disk1 = gdspy.Round((0, 0), metaatomRadius[0], tolerance=0.001)
disk2 = gdspy.Round((0, 0), metaatomRadius[1], tolerance=0.001)
disk3 = gdspy.Round((0, 0), metaatomRadius[2], tolerance=0.001)
disk4 = gdspy.Round((0, 0), metaatomRadius[3], tolerance=0.001)
disk5 = gdspy.Round((0, 0), metaatomRadius[4], tolerance=0.001)
disk6 = gdspy.Round((0, 0), metaatomRadius[5], tolerance=0.001)
disk7 = gdspy.Round((0, 0), metaatomRadius[6], tolerance=0.001)
disk8 = gdspy.Round((0, 0), metaatomRadius[7], tolerance=0.001)

In [ ]:
MA1cell=lib.new_cell("metaatom-1")
MA1cell.add(disk1)

MA2cell=lib.new_cell("metaatom-2")
MA2cell.add(disk2)

MA3cell=lib.new_cell("metaatom-3")
MA3cell.add(disk3)

MA4cell=lib.new_cell("metaatom-4")
MA4cell.add(disk4)

MA5cell=lib.new_cell("metaatom-5")
MA5cell.add(disk5)

MA6cell=lib.new_cell("metaatom-6")
MA6cell.add(disk6)

MA7cell=lib.new_cell("metaatom-7")
MA7cell.add(disk7)

MA8cell=lib.new_cell("metaatom-8")
MA8cell.add(disk8)

MAcell = [MA1cell, MA2cell, MA3cell, MA4cell, MA5cell, MA6cell, MA7cell, MA8cell]

# lib.add([MA1cell, MA2cell, MA3cell,MA4cell,MA5cell,MA6cell,MA7cell,MA8cell])
# lib.write_gds('Metasurface1.gds')
# gdspy.LayoutViewer()

In [ ]:
#CONVERTING PHASE MAP to RADIUS MAP
def phaseToCellMap(center, phase, phaseValue, MAcell):
    for i in range(len(phaseValue)):
        if phase<phaseValue[i]+phaseStep/2 and phase>=phaseValue[i]-phaseStep/2:
            return gdspy.CellReference(MAcell[i], center)
        elif phase>max(phaseValue)+phaseStep/2:
            return gdspy.CellReference(MAcell[0], center)

In [ ]:
#GENERATING SPATIAL X,Y GRID
def spatialMapGenerator(periodX, periodY, phaseMap):
    spatialMap = [[[periodX*x, periodY*y] for x in range(len(phaseMap))] for y in range(len(phaseMap[0]))]
    return spatialMap

In [ ]:
#PUTTING IDEAL PHASE MAP ON A GRID (discretization)
spatialMap = spatialMapGenerator(periodX, periodY, phaseMap)
spatialMap

In [ ]:
#CREATING THE LIBRARY of META-ATOMS
#function to generate a circle of radius 'r' at position 'center'
# def phaseDiskPoly(phase, phaseToRadiusMap, center):
#     r = phaseToDiskRadius(phase, phaseToRadiusMap)
#     disk = gdspy.Round(center,r) #!change round; return the cell reference, #documention on self reference
#     return disk

In [ ]:
#function to generate 2D map of circles (metasurface)
def metalensGenerator(phaseMap, phaseToCellMap, spatialMap):
    metasurface = []
    for x in range(len(phaseMap)):
        for y in range(len(phaseMap[0])):
            metasurface.append(phaseToCellMap(spatialMap[x][y], phaseMap[x][y], phaseValue, MAcell))
    return metasurface
        

In [ ]:
metalens = metalensGenerator(phaseMap, phaseToCellMap, spatialMap)

In [ ]:
metalens

In [ ]:
Lens=gdspy.Cell("Lens-1")
Lens.add(metalens)

In [ ]:
#writing gds file
gdspy.current_library = gdspy.GdsLibrary()
gdspy.current_library.add(Lens)
# gdspy.current_library.add(top)
# top.add(gdspy.CellReference(Lens))
# top.add(gdspy.CellReference(Lens, (2*mm,2*mm)))
gdspy.write_gds('20200728_metasurface0p5mm_scaled5.gds')